In [2]:
import pandas as pd 
import sqlite3

In [128]:
# Connect to or create a new database file called 'superstore_database.db'

conn = sqlite3.connect('superstore_database.db')

In [129]:
# Create a cursor

cursor = conn.cursor()

In [10]:
db = pd.read_csv("Sample-Superstore.csv", encoding='latin1')

In [31]:
db.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,purchase_price
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,151.9368
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,317.1740
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,10.7457
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,114.9093
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,12.4422


In [18]:
db["purchase_price "] =(db["Sales"] + db["Profit"])/db.Quantity

In [19]:
# SALES
conn.execute(
    """
    CREATE TABLE sales (
        SalesID INTEGER ,
        OrderID TEXT NOT NULL,
        ProductID TEXT NOT NULL,
        Sales REAL,
        Quantity INTEGER,
        Discount REAL,
        Profit REAL,
        PRIMARY KEY(SalesID),
        FOREIGN KEY(OrderID) REFERENCES orders(OrderID),
        FOREIGN KEY(ProductID) REFERENCES products(ProductID)
        );
     """
)

# ORDERS
conn.execute(
    """
    CREATE TABLE orders (
        OrderID TEXT,
        OrderDate TEXT,
        ShipDate TEXT,
        ShipMode TEXT,
        CustomerID TEXT,
        PRIMARY KEY(OrderID),
        FOREIGN KEY(CustomerID) REFERENCES customers(CustomerID)
        );
    """
)

# CUSTOMERS
conn.execute(
    """
    CREATE TABLE customers (
        CustomerID TEXT PRIMARY KEY,
        CustomerName TEXT,
        Segment TEXT,
        Country TEXT,
        City TEXT,
        State TEXT,
        PostalCode TEXT,
        Region TEXT
        );
    """
)

# PRODUCTS
conn.execute(
    """
    CREATE TABLE products (
        ProductID TEXT PRIMARY KEY,
        PurchasePrice REAL
        Category TEXT,
        SubCategory TEXT,
        ProductName TEXT,
        );
    """
)

In [76]:
#conn.execute('DROP TABLE IF EXISTS products')
# PRODUCTS
conn.execute(
    """
    CREATE TABLE products (
        ProductID TEXT PRIMARY KEY,
        PurchasePrice REAL,
        Category TEXT,
        SubCategory TEXT,
        ProductName TEXT
        );
    """
)



In [24]:
#Create Sales DB 
sales = db[["Order ID","Product ID", "Sales", "Quantity", "Discount", "Profit"]]


RangeIndex(start=0, stop=9994, step=1)

In [28]:
# Create index column 
sales=sales.rename_axis("SalesID").reset_index()

In [29]:
#Chanege the column name to correct 
sales.columns = [
    "SalesID",
    "OrderID",
    "ProductID",
    "Sales",
    "Quantity",
    "Discount",
    "Profit"
]

In [30]:
sales.head()

,SalesID,OrderID,ProductID,Sales,Quantity,Discount,Profit
0,0,CA-2016-152156,FUR-BO-10001798,261.9600,2,0.00,41.9136
1,1,CA-2016-152156,FUR-CH-10000454,731.9400,3,0.00,219.5820
2,2,CA-2016-138688,OFF-LA-10000240,14.6200,2,0.00,6.8714
3,3,US-2015-108966,FUR-TA-10000577,957.5775,5,0.45,-383.0310
4,4,US-2015-108966,OFF-ST-10000760,22.3680,2,0.20,2.5164


In [33]:
#sales.to_sql('sales', conn, if_exists='append', index=False)
pd.read_sql("SELECT * FROM sales LIMIT 5", conn)

,SalesID,OrderID,ProductID,Sales,Quantity,Discount,Profit
0,0,CA-2016-152156,FUR-BO-10001798,261.9600,2,0.00,41.9136
1,1,CA-2016-152156,FUR-CH-10000454,731.9400,3,0.00,219.5820
2,2,CA-2016-138688,OFF-LA-10000240,14.6200,2,0.00,6.8714
3,3,US-2015-108966,FUR-TA-10000577,957.5775,5,0.45,-383.0310
4,4,US-2015-108966,OFF-ST-10000760,22.3680,2,0.20,2.5164


#### Chane the columns names 

In [55]:
db.columns = ['RowID', 'OrderID', 'OrderDate', 'ShipDate', 'ShipMode',
       'CustomerID', 'CustomerName', 'Segment', 'Country', 'City', 'State',
       'PostalCode', 'Region', 'ProductID', 'Category', 'SubCategory',
       'ProductName', 'Sales', 'Quantity', 'Discount', 'Profit',
       'PurchasePrice']

##### Create other db and tables

In [56]:
orders=db[['OrderID', 'OrderDate', 'ShipDate', 'ShipMode',
       'CustomerID']]
customers = db[['CustomerID', 'CustomerName', 'Segment', 'Country', 'City', 'State',
       'PostalCode', 'Region',]]
products = db[['ProductID', 'Category', 'SubCategory',
       'ProductName','PurchasePrice']]

In [77]:
# Remove duplicates from orders and customers DataFrames
#orders = orders.drop_duplicates(subset=['OrderID'])
#customers = customers.drop_duplicates(subset=['CustomerID'])
products = products.drop_duplicates(subset=['ProductID'])

# Insert the data into the tables using pandas to_sql method
#orders.to_sql('orders', conn, if_exists='append', index=False)
#customers.to_sql('customers', conn, if_exists='append', index=False)
products.to_sql('products', conn, if_exists='append', index=False)


1818

In [59]:
orders.duplicated().sum()

0

In [80]:
pd.read_sql("select * from products limit 5", conn)

,ProductID,PurchasePrice,Category,SubCategory,ProductName
0,FUR-BO-10001798,151.9368,Furniture,Bookcases,Bush Somerset Collection Bookcase
1,FUR-CH-10000454,317.1740,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,..."
2,OFF-LA-10000240,10.7457,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...
3,FUR-TA-10000577,114.9093,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table
4,OFF-ST-10000760,12.4422,Office Supplies,Storage,Eldon Fold 'N Roll Cart System


In [85]:
cursor.execute("select * from products limit 5")
product_result = cursor.fetchall()
product_result

[('FUR-BO-10001798',
  151.9368,
  'Furniture',
  'Bookcases',
  'Bush Somerset Collection Bookcase'),
 ('FUR-CH-10000454',
  317.17400000000004,
  'Furniture',
  'Chairs',
  'Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back'),
 ('OFF-LA-10000240',
  10.7457,
  'Office Supplies',
  'Labels',
  'Self-Adhesive Address Labels for Typewriters by Universal'),
 ('FUR-TA-10000577',
  114.90929999999999,
  'Furniture',
  'Tables',
  'Bretford CR4500 Series Slim Rectangular Table'),
 ('OFF-ST-10000760',
  12.4422,
  'Office Supplies',
  'Storage',
  "Eldon Fold 'N Roll Cart System")]

In [131]:
# Using pandas read_sql to check the existing tables
tables_df = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
print("Tables using pandas.read_sql:")
print(tables_df)

# Using cursor.execute to check the existing tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables_result = cursor.fetchall()
print("Tables using cursor.execute:")
for table in tables_result:
    print(table[0])

Tables using pandas.read_sql:
        name
0      sales
1     orders
2  customers
3      order
4   products
Tables using cursor.execute:
sales
orders
customers
order
products


In [133]:
#conn.execute("drop table if exists order")
#cursor.execute('DROP TABLE IF EXISTS order')

conn.commit()  # Roll back any pending transaction


In [134]:
conn.close()  # Close the existing connection
# Create a new connection and drop the tables
conn = sqlite3.connect('superstore.db')
cursor = conn.cursor()
cursor.execute('DROP TABLE IF EXISTS order')
conn.commit()
conn.close()



OperationalError: near "order": syntax error